# Deep Learning in Neuroscience by Edgar Y. Walker

## Preparing the environment

### NOTE: Please run this section at the very beginning of the first session!

Before we get to dive in and learn how deep learning is used in neuroscience and get your first neural predictive model trained, we need to install some prerequisite packages and download some neuronal data!

### Getting the code

We are going to primarily use [PyTorch](https://pytorch.org) to build, train and evaluate our deep learning models and I am going to assume some familiarity with PyTorch already.

Also to be able to handle the dataset containing neuronal activities, we are going to make our life easier by using a few existing libraries. I have prepared a library called [lviv2021](https://github.com/eywalker/lviv2021). This library has a dependency on [neuralpredictors](https://github.com/sinzlab/neuralpredictors), which is a collection of PyTorch layers, tools and other utilities that would prove helpful to train networks to predict neuronal responses.

Let's go ahead and install this inside the Colab environment.

In [1]:
# Install PyTorch dependency
!pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
    
# Install 
!pip3 install git+https://github.com/eywalker/lviv-2021.git

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1.9 MB 7.5 MB/s 
  Cloning https://github.com/sinzlab/neuralpredictors.git (to revision v0.0) to /tmp/pip-req-build-v6c91am7
  Running command git clone -q https://github.com/sinzlab/neuralpredictors.git /tmp/pip-req-build-v6c91am7
  Running command git checkout -b v0.0 --track origin/v0.0
  Switched to a new branch 'v0.0'
  Branch 'v0.0' set up to track remote branch 'v0.0' from 'origin'.
  Created wheel for neuralpredictors: filename=neuralpredictors-0.0.3-py3-none-any.whl size=56066 sha256=8b5329ddfdd5504177e8c77abcf15b941bb1abd1933fc223750bf1ff7ba86c5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-a20585dv/wheels/2b/55/61/c9d55e5d782d397dde737beea8c31e670fd96ca541b7fd3250
Successfully built neuralpredictors


### Getting the dataset

We are going to use the dataset made available for our recent paper [Lurz et al. ICLR 2021](https://github.com/sinzlab/Lurz_2020_code), predicting responses of mouse visual cortex to natural images. 

The dataset is relatively large, so please be sure to run the following at the very beginning of the session. We are going to first spend some time learning the basics of computational neuroscience in the study of system identification. It would be best that you let the download take place while we go over the neursocience primer so that it will be ready when we come back here to get our hands dirty!

To download the data, simply execute the following cell, and let it run till completion.

In [1]:
!mkdir /data
!git clone https://gin.g-node.org/cajal/Lurz2020.git /data

mkdir: cannot create directory ‘/data’: File exists
Cloning into '/data'...
remote: Enumerating objects: 23150, done.                                            
remote: Counting objects: 100% (23150/23150), done.                                  
remote: Compressing objects: 100% (11714/11714), done.                                  
remote: Total 23150 (delta 11436), reused 23134 (delta 11434)    
Receiving objects: 100% (23150/23150), 252.05 MiB | 506.00 KiB/s, done.
Resolving deltas: 100% (11436/11436), done.
Checking out files: 100% (24035/24035), done.


## Install PyTorch and other dependencies

In [2]:
!pip3 install git+https://github.com/ZhuokunDing/Iviv-2021

  Cloning https://github.com/ZhuokunDing/Iviv-2021 to /tmp/pip-req-build-gdwm0nz6
  Running command git clone -q https://github.com/ZhuokunDing/Iviv-2021 /tmp/pip-req-build-gdwm0nz6
  Created wheel for iviv: filename=iviv-0.1-py3-none-any.whl size=15184 sha256=1959ec14515ffaad19ea7cd05c572670e5174234dc115c9a0f2c9ad5e71705fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-3geqjt09/wheels/6e/8a/fb/883bcd3dc8880ce76d927912d45aa5a712c4ca35d537ca168d
Successfully built iviv


## Getting our hands dirty

Now that you have been primed with just enough background neuroscience, let's get our hand dirty and try to build our first neural predictive models. First thing fi

## Download data
Approximate download time: ~5 mins

In [1]:
!mkdir /data
!git clone https://gin.g-node.org/cajal/Lurz2020.git /data

mkdir: cannot create directory ‘/data’: File exists
Cloning into '/data'...
remote: Enumerating objects: 23150, done.                                            
remote: Counting objects: 100% (23150/23150), done.                                  
remote: Compressing objects: 100% (11714/11714), done.                                  
remote: Total 23150 (delta 11436), reused 23134 (delta 11434)    
Receiving objects: 100% (23150/23150), 252.05 MiB | 506.00 KiB/s, done.
Resolving deltas: 100% (11436/11436), done.
Checking out files: 100% (24035/24035), done.


## Load data

In [1]:
from lviv.dataset import load_dataset

In [52]:
neuron_n = None
image_n = None
dataloaders = load_dataset(path = '/data/static20457-5-9-preproc0', batch_size=60, seed=1234, image_n=image_n, neuron_n=neuron_n)

In [53]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f621bda7970>,
 'validation': <torch.utils.data.dataloader.DataLoader at 0x7f621bda7d90>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f621bda7dc0>}

In [54]:
train_loader = dataloaders['train']

In [55]:
len(train_loader.sampler)

4472

In [56]:
images, responses = next(iter(train_loader))

In [15]:
images.shape

torch.Size([60, 1, 36, 64])

In [16]:
responses.shape

torch.Size([60, 5335])

## Build model

In [117]:
import torch
from torch import nn
from collections import OrderedDict
from typing import Iterable

class Linear(nn.Module):
    def __init__(
        self,
        input_height,
        input_width,
        n_neurons,
        momentum=0.1,
        init_std=0.01,
        gamma=0
    ):

        """
        Args:
            input_channels:     Integer, number of input channels as in
            hidden_channels:    Number of hidden channels (i.e feature maps) in each hidden layer
            input_kern:     kernel size of the first layer (i.e. the input layer)
            hidden_kern:    kernel size of each hidden layer's kernel
            layers:         number of layers
            momentum:       BN momentum
            pad_input:      Boolean, if True, applies zero padding to all convolutions
            batch_norm:     Boolean, if True appends a BN layer after each convolutional layer
            hidden_dilation:   If set to > 1, will apply dilated convs for all hidden layers
            linear:         Boolean, if True, remove all nonlinearity in the model
            readout:
        """
        super().__init__()
        self.bn = nn.BatchNorm2d(1, momentum=momentum, affine=False)
        self.linear = nn.Linear(input_height * input_width, n_neurons)
        self.gamma = gamma
        self.init_std = init_std
        self.initialize()
        

    def initialize(self, std=None):
        if std is None:
            std = self.init_std
        nn.init.normal_(self.linear.weight.data, std=std)


    def forward(self, x):
        #x = self.bn(x)
        #(x - 0.1460)
        x = self.linear(x.flatten(1))
        return nn.functional.elu(x) + 1

    def regularizer(self):
        return self.gamma * self.linear.weight.abs().sum()


In [93]:
model.linear.weight.std()

tensor(0.0505, device='cuda:0', grad_fn=<StdBackward0>)

In [66]:
import torch
from torch import nn
from collections import OrderedDict
from typing import Iterable

class CNN(nn.Module):
    def __init__(
        self,
        input_height,
        input_width,
        n_neurons,
        input_channels,
        hidden_channels,
        input_kern,
        hidden_kern,
        layers=3,
        momentum=0.1,
        pad_input=True,
        batch_norm=True,
        hidden_dilation=1,
        linear=False,
        readout='fc',
        gamma=0
    ):

        """
        Args:
            input_channels:     Integer, number of input channels as in
            hidden_channels:    Number of hidden channels (i.e feature maps) in each hidden layer
            input_kern:     kernel size of the first layer (i.e. the input layer)
            hidden_kern:    kernel size of each hidden layer's kernel
            layers:         number of layers
            momentum:       BN momentum
            pad_input:      Boolean, if True, applies zero padding to all convolutions
            batch_norm:     Boolean, if True appends a BN layer after each convolutional layer
            hidden_dilation:   If set to > 1, will apply dilated convs for all hidden layers
            linear:         Boolean, if True, remove all nonlinearity in the model
            readout:
        """
        super().__init__()
        self.layers = layers
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.core = nn.Sequential()
        self.readout_type = readout
        self.n_neurons = n_neurons
        self.gamma = gamma


        # Core: --- first layer
        layer = OrderedDict()
        layer["conv"] = nn.Conv2d(
            input_channels, hidden_channels, input_kern, padding=input_kern // 2 if pad_input else 0, bias=False
        )
        if batch_norm:
            layer["norm"] = nn.BatchNorm2d(hidden_channels, momentum=momentum)
        if not linear:
            layer["nonlin"] = nn.ELU(inplace=True)
        self.core.add_module("layer0", nn.Sequential(layer))
        if not isinstance(hidden_kern, Iterable):
            hidden_kern = [hidden_kern] * (self.layers - 1)

        # Core: --- other layers
        for l in range(1, self.layers):
            layer = OrderedDict()
            hidden_padding = ((hidden_kern[l - 1] - 1) * hidden_dilation + 1) // 2
            layer["conv"] = nn.Conv2d(
                hidden_channels,
                hidden_channels,
                hidden_kern[l - 1],
                padding=hidden_padding,
                bias=False,
                dilation=hidden_dilation,
            )
            if batch_norm:
                layer["norm"] = nn.BatchNorm2d(hidden_channels, momentum=momentum)
            if not linear:
                layer["nonlin"] = nn.ELU(inplace=True)
            self.core.add_module("layer{}".format(l), nn.Sequential(layer))
        self.apply(self.init_conv)
        
        # Readout
        ## fully connected readout
        if readout == 'fc':
            in_dim = self.core_channels * input_height * input_width
            self.readout = nn.Linear(in_dim, n_neurons, bias=True)
        ## spatial separable readout
        elif readout == 'spatial':
            self.spatial = nn.Parameter(torch.Tensor(n_neurons, input_width, input_height))
            self.features = nn.Parameter(torch.Tensor(n_neurons, self.core_channels))
            self.readout_bias = nn.Parameter(torch.Tensor(n_neurons))
            
    def forward(self, input_):
        ret = []
        for l, feat in enumerate(self.core):
            input_ = feat(input_)
            ret.append(input_)
        core_out = torch.cat(ret, dim=1)
        
        if self.readout_type == 'fc':
            core_out = core_out.view([core_out.shape[0], -1])
            readout_out = nn.functional.elu(self.readout(core_out)) + 1
        elif self.readout_type == 'spatial':
            readout_out = torch.einsum("ncwh,owh->nco", core_out, self.spatial)
            readout_out = torch.einsum("nco,oc->no", readout_out, self.features)
            readout_out = readout_out + self.readout_bias
            readout_out = nn.functional.elu(readout_out) + 1
        return readout_out

    def regularizer(self):
        if self.readout_type == 'fc':
            return self.readout.weight.abs().sum() * self.gamma
        elif self.readout_type == 'spatial':
            return (
            self.spatial.view(self.n_neurons, -1).abs().sum(dim=1, keepdim=True)
            * self.features.view(self.n_neurons, -1).abs().sum(dim=1)
        ).sum() * self.gamma

    @property
    def core_channels(self):
        return len(self.core) * self.hidden_channels
    
    @staticmethod
    def init_conv(m):
        if isinstance(m, nn.Conv2d):
            nn.init.xavier_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0)
                
    def initialize_readout(self):
        if self.readout_type == 'full':
            nn.init.xavier_normal_(self.readout.weight.data)
            self.readout.weight.data.fill_(0)
        elif self.readout_type == 'spatial':
            self.spatial.data.normal_(0, 1e-3)
            self.features.data.normal_(0, 1e-3)
            if self.readout_bias is not None:
                self.readout_bias.data.fill_(0)
                
    def initialize(self):
        self.core.apply(self.init_conv)
        self.initialize_readout()
            


In [8]:
torch.cuda.is_available()

True

In [68]:
neuron_n = 5335
model_config = {
    'input_height': 64,
    'input_width': 36,
    'n_neurons': neuron_n,
    'input_channels': 1,
    'hidden_channels': 64,
    'input_kern': 15,
    'hidden_kern': 13,
    'layers': 4,
    'readout': 'spatial',
    'gamma': 0.001,
}
model = CNN(**model_config)
model.initialize()

# be sure to place it on the GPU!
model.cuda()

CNN(
  (core): Sequential(
    (layer0): Sequential(
      (conv): Conv2d(1, 64, kernel_size=(15, 15), stride=(1, 1), padding=(7, 7), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): ELU(alpha=1.0, inplace=True)
    )
    (layer1): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(13, 13), stride=(1, 1), padding=(6, 6), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): ELU(alpha=1.0, inplace=True)
    )
    (layer2): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(13, 13), stride=(1, 1), padding=(6, 6), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): ELU(alpha=1.0, inplace=True)
    )
    (layer3): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(13, 13), stride=(1, 1), padding=(6, 6), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affi

In [62]:
from lviv.models import build_lurz2020_model
model_config = {'init_mu_range': 0.55,
                'init_sigma': 0.4,
                'input_kern': 15,
                'hidden_kern': 13,
                'gamma_input': 1.0,
                'grid_mean_predictor': {'type': 'cortex',
                                        'input_dimensions': 2,
                                        'hidden_layers': 0,
                                        'hidden_features': 0,
                                        'final_tanh': False},
                'gamma_readout': 2.439,
                'linear': False}

model = build_lurz2020_model(**model_config, dataloaders=dataloaders, seed=1234)

## Build trainer

In [108]:
from lviv.trainers import train_model

trainer_config = {'track_training': True,
                  'detach_core': False}

## Model training
Approximate training time: ~15 mins

In [118]:
model = Linear(input_height=64, input_width=36, n_neurons=5335, gamma=0.000, init_std=0.001)

In [ ]:
score, output, model_state = train_model(model=model, dataloader=dataloaders, seed=1, lr_init=1e-6, **trainer_config)

correlation -8.0261256e-05
poisson_loss 9588915.0


Epoch 1: 100%|██████████| 75/75 [00:00<00:00, 83.59it/s]


[001|00/05] ---> 0.005938975140452385
correlation 0.005938975
poisson_loss 3507831.2


Epoch 2: 100%|██████████| 75/75 [00:00<00:00, 84.14it/s]


[002|00/05] ---> 0.010033603757619858
correlation 0.010033604
poisson_loss 3307697.5


...

Epoch 7: 100%|██████████| 75/75 [00:00<00:00, 79.99it/s]


[007|00/05] ---> 0.02730812504887581
correlation 0.027308125
poisson_loss 2877410.0


Epoch 8: 100%|██████████| 75/75 [00:00<00:00, 82.01it/s]


[008|00/05] ---> 0.02922097034752369
correlation 0.02922097
poisson_loss 2835198.5


Epoch 9:  83%|████████▎ | 62/75 [00:00<00:00, 84.83it/s]

In [103]:
model.bn.bias

Parameter containing:
tensor([0.1460], device='cuda:0', requires_grad=True)

In [104]:
model.bn.weight

Parameter containing:
tensor([0.2764], device='cuda:0', requires_grad=True)

In [105]:
model.bn??

Signature:      model.bn(*input, **kwargs)
Type:           BatchNorm2d
String form:    BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
File:           /usr/local/lib/python3.8/dist-packages/torch/nn/modules/batchnorm.py
Source:        
class BatchNorm2d(_BatchNorm):
    r"""Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs
    with additional channel dimension) as described in the paper
    `Batch Normalization: Accelerating Deep Network Training by Reducing
    Internal Covariate Shift <https://arxiv.org/abs/1502.03167>`__ .

    .. math::

        y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta

    The mean and standard-deviation are calculated per-dimension over
    the mini-batches and :math:`\gamma` and :math:`\beta` are learnable parameter vectors
    of size `C` (where `C` is the input size). By default, the elements of :math:`\gamma` are set
    to 1 and the elements of :math:`\beta` are set 